### Script to generate summaries using chunking based Pegasus approach

In [56]:
dataset = "IN-Abs" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext
output_path = "/Users/lakke21/Downloads/summarization-aacl/abstractive/Legal-Pegasus/output"


In [57]:
import pandas as pd
import numpy as np
import glob
import sys
sys.path.insert(0, '../')
from utilities import *
import os
import nltk

In [58]:
def get_root_path1():
    '''
    function to get root path of dataset

    change the path variable to the path of the dataset
    '''
    path = "/Users/lakke21/Downloads/dataset"
    return path

In [59]:
def get_summary_data1(dataset, train):
    '''
    function to get names, documents, and summaries

    change the path variable to the path of the dataset
    
    if dataset == "N2":
        path = get_root_path1() + '/N2/Full-Text/India'
        all_files = glob.glob(path + "/*.txt")

        data_source = []
        names = []
        for filename in all_files:
            with open(filename, 'r') as f: 
                p = filename.rfind("/")
#                 print(filename[p+1:])
                names.append(filename[p+1:])
                a = f.read()
                data_source.append(a)
        return names, data_source, []'''
    
    path = get_root_path1() + '/' + dataset + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path1() + '/' + dataset + '/' + train + '-data/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

In [60]:
def get_req_len_dict1(dataset, istrain):
    '''
    function to required length data for each summary

    change the path variable to the path to Summary_Length_India.txt/ Summary_Length_Uk.txt file
    '''
    
    '''if dataset == "N2":
        f = open(get_root_path1() + "/N2/Summary_Length_India.txt", "r")
        a = (f.read())
        a = a.split("\n")
        dict_names = {}
        for i in a:
            b = i.split("	")
            dict_names[b[0] + ".txt"] = int(b[1])
        return dict_names'''
    
    f = open(get_root_path1() + "/IN-Abs/test-data/stats-IN-test.txt", "r")
    a = (f.read())
    a = a.split("\n")
    dict_names = {}
    for i in a:
        b = i.split("	")
        try:
            tp = int(b[2])
            dict_names[b[0]] = tp
        except:
            print(b)
    return dict_names  

In [61]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [62]:
#Reading the test documents
names, data_source, data_summary = get_summary_data1(dataset, "test")
print(len(names))
print(len(data_source))
print(len(data_summary))
len_dic = dict_names = get_req_len_dict1(dataset, "test")  

100
100
100
['']


In [63]:
device = "cpu"

In [64]:
# Loading Model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments, PegasusTokenizerFast

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus").to(device)
#model_path = '/Users/lakke21/Downloads/summarization-aacl/abstractive/Legal-Pegasus/ouput_model'  # Ensure this is the path where your model is saved
#model = PegasusForConditionalGeneration.from_pretrained(model_path)
#tokenizer = PegasusTokenizerFast.from_pretrained(model_path)


In [65]:
def summerize(text, max_len, min_len):
    '''
    Function to generate summary using Pegasus
    input:  nested_sentences - chunks
            max_l - Maximum length
            min_l - Minimum length
    output: document summary
    '''
    try:
        input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=512,truncation=True).to(device)
        summary_ids = model.generate(input_tokenized,
                                          num_beams=9,
                                          length_penalty=0.1,
                                          min_length=min_len,
                                          max_length=max_len,
                                    )
        summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
        return summary
    except:
        return ""

In [66]:
def summerize_doc(nested_sentences, p):
    '''
    Function to generate summary using chunking based Pegasus
    input:  nested_sentences - chunks
            p - Number of words in summaries per word in the document
    output: document summary
    '''
    device = 'cpu'
    result = []
    for nested in nested_sentences:
        l = int(p * len(nested.split(" ")))
        max_len = l
        min_len = l-5
        result.append(summerize(nested, max_len, min_len))
    return result

In [67]:
'''done_files = glob.glob(output_path + "*.txt")
done_files = [i[i.rfind("/")+1:] for i in done_files]'''

new_output_path = "/Users/lakke21/Downloads/summarization-aacl/abstractive/Legal-Pegasus/output"

# Ensure the new output directory exists
if not os.path.exists(new_output_path):
    os.makedirs(new_output_path)

In [68]:
'''# main loop to generate and save summaries of each document in the test dataset
#for i in range(len(data_source)):
for i in range(1):
    done_files = glob.glob(output_path + "*.txt")
    done_files = [i[i.rfind("/")+1:] for i in done_files]
    name = names[i]'''
for i in range(1):
    # Use the new output path for checking existing files
    done_files = glob.glob(os.path.join(new_output_path, "*.txt"))
    #done_files = [os.path.basename(f) for f in done_files]
    name = names[i]
    if name in done_files:continue
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = dict_names[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = ", ")
    
    nested = nest_sentences(doc,512)
    p = float(req_len/input_len)
    print(p)
    abs_summ = summerize_doc(nested,p)
    abs_summ = " ".join(abs_summ)
    print(len((abs_summ.split(" "))))
    
    if len(abs_summ.split(" ")) > req_len:
        abs_summ = abs_summ.split(" ")
        abs_summ = abs_summ[:req_len]
        abs_summ = " ".join(abs_summ)
    print(len((abs_summ.split(" "))))
    summary_file_path = os.path.join(new_output_path, name)
    #path = new_output_path + name
    file = open(summary_file_path,'w')
    file.write(abs_summ)
    file.close()
#     break'''

0: 6728.txt - 830 : 436, 0.5253012048192771
403
403


In [69]:
'''import os
import glob

# Update this to your desired new output directory
new_output_path = "/Users/lakke21/Downloads/summarization-aacl/abstractive/Legal-Pegasus/output"

# Ensure the new output directory exists
if not os.path.exists(new_output_path):
    os.makedirs(new_output_path)

# main loop to generate and save summaries of each document in the test dataset
for i in range(1):
    # Use the new output path for checking existing files
    done_files = glob.glob(os.path.join(new_output_path, "*.txt"))
    done_files = [os.path.basename(f) for f in done_files]
    name = names[i]

    print(f"Checking if summary for {name} exists in the new location...")

    if name in done_files:
        print(f"Summary for {name} already exists. Skipping...")
        continue
    
    # ... (rest of your code for generating summary)

    # Use the new output path for saving the summary
    summary_file_path = os.path.join(new_output_path, name)

    print(f"Saving summary for {name} at {summary_file_path}")

    with open(summary_file_path, 'w') as file:
        file.write(abs_summ)

    print(f"Summary for {name} saved successfully.")'''


'import os\nimport glob\n\n# Update this to your desired new output directory\nnew_output_path = "/Users/lakke21/Downloads/summarization-aacl/abstractive/Legal-Pegasus/output"\n\n# Ensure the new output directory exists\nif not os.path.exists(new_output_path):\n    os.makedirs(new_output_path)\n\n# main loop to generate and save summaries of each document in the test dataset\nfor i in range(1):\n    # Use the new output path for checking existing files\n    done_files = glob.glob(os.path.join(new_output_path, "*.txt"))\n    done_files = [os.path.basename(f) for f in done_files]\n    name = names[i]\n\n    print(f"Checking if summary for {name} exists in the new location...")\n\n    if name in done_files:\n        print(f"Summary for {name} already exists. Skipping...")\n        continue\n    \n    # ... (rest of your code for generating summary)\n\n    # Use the new output path for saving the summary\n    summary_file_path = os.path.join(new_output_path, name)\n\n    print(f"Saving sum